In [3]:
import torch
import transformers

from anki_ai.domain.model import Deck

In [4]:
deck = Deck("default")

In [5]:
deck.read_txt("../data/Selected Notes.txt")
len(deck)

2024-08-25 09:35:51.861 | WARNING  | anki_ai.domain.model:read_txt:48 - Was not able to process line 0: #separator:tab

2024-08-25 09:35:51.862 | WARNING  | anki_ai.domain.model:read_txt:48 - Was not able to process line 1: #html:true

2024-08-25 09:35:51.863 | WARNING  | anki_ai.domain.model:read_txt:48 - Was not able to process line 2: #tags column:6

2024-08-25 09:35:51.864 | WARNING  | anki_ai.domain.model:read_txt:48 - Was not able to process line 366: "<div>What pandas DataFrame method and arguments can be used to create a histogram with 50 bins for the 'age' column of a DataFrame called 'users'?</div>

2024-08-25 09:35:51.865 | WARNING  | anki_ai.domain.model:read_txt:48 - Was not able to process line 378: "What is the command to return <b><font color=""#ef2929"">all unique values</font></b> for a variable?"	"<center><table class=""highlighttable""><tbody><tr><td><div class=""linenodiv"" style=""background-color: #f0f0f0; padding-right: 10px""><pre style=""line-height: 125%"">1<

2241

In [6]:
# model_id = "meta-llama/Meta-Llama-3.1-8B"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    max_length=300,
    truncation=True,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
prompt = f"""Make this Anki note concise, simple, straightforward, and distinct.

Reply with this format:

Front: [front section of card 1]
Back: [back section of card 1]

Here is the note:

Front: {deck[0].front}
Back: {deck[0].back}
"""
reply = pipeline(prompt)
print(reply[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Make this Anki note concise, simple, straightforward, and distinct.

Reply with this format:

Front: [front section of card 1]
Back: [back section of card 1]

Here is the note:

Front: What command does create a soft link?
Back: ```bash<br>$ ln -s &lt;file_name&gt; &lt;link_name&gt;<br>```
```bash<br>$ ln -s &lt;file_name&gt; &lt;link_name&gt;<br>```

Here is the revised note:

Front: Create soft link command
Back: `ln -s`


Note: I removed the unnecessary extra backticks and the unnecessary example, and made the front and back sections concise and distinct. Let me know if you'd like any further changes. 

Let me know if you want me to make any changes. 

Here is the revised note:

Front: Create soft link command
Back: `ln -s`


I also removed the unnecessary extra backticks and the unnecessary example, and made the front and back sections concise and distinct. Let me know if you'd like any further changes. 

Here is the revised note:

Front: Create soft link command
Back: `ln -s`


Th

In [12]:
del pipeline

### Use chat template

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
system_msg = """You are an expert in building Anki cards.

The user will share an existing Anki note with you. Your job is to edit it to be more concise, simple, straightforward, and distinct.

You reply in the following format:
   
Front: [front section of card 1]
Back: [back section of card 1]
"""

user_msg = f"""Front: {deck[0].front}
Back: {deck[0].back}
"""

messages = [
   {"role": "system", "content": system_msg},
   {"role": "user", "content": user_msg},
]

tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(tokenized_chat, max_new_tokens=150) 
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert in building Anki cards.

The user will share an existing Anki note with you. Your job is to edit it to be more concise, simple, straightforward, and distinct.

You reply in the following format:
   
Front: [front section of card 1]
Back: [back section of card 1]<|eot_id|><|start_header_id|>user<|end_header_id|>

Front: What command does create a soft link?
Back: ```bash<br>$ ln -s &lt;file_name&gt; &lt;link_name&gt;<br>```<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Front: Create a soft link
Back: `ln -s <file_name> <link_name>`<|eot_id|>


In [ ]:
"""Create [number of cards] concise, simple, straightforward and distinct Anki cards to study the following article, each with a front and back. Avoid repeating the content in the front on the back of the card. In particular, if the front is a question and the back an answer, avoid repeating the phrasing of the question as the initial part of the answer. Avoid explicitly referring to the author or the article in the cards, and instead treat the article as factual and independent of the author. Use the following format:

1. Front: [front section of card 1]
Back: [back section of card 1]

2. Front: [front section of card 2]
Back: [back section of card 2]

... and so on.

Here is the article:

"[Article]"""